In [1]:
from fastai.text import *
import pandas as pd
from path import Path as path

In [2]:
enfj = pd.read_csv("enfj.csv", index_col = 0)
infp = pd.read_csv("infp.csv", index_col = 0)
enfj.head()

,subreddit,text
0,enfj,I really like doing things for my friends and ...
1,enfj,I think everyone in this world would want to f...
2,enfj,Sometimes I feel like people don't appreciate ...
3,enfj,I love making other people happy -- appreciate...
4,enfj,Sometimes. And when my expectations aren't met...


In [3]:
df = enfj.append(infp)
path2 = path("model/").mkdir_p()
df.to_csv("model/enfj_infp.csv", index=None)

In [4]:
# Language model data
data_lm = TextLMDataBunch.from_csv(path2, 'enfj_infp.csv', num_workers = 0)
# Classifier model data
data_clas = TextClasDataBunch.from_csv(path2, 'enfj_infp.csv', vocab=data_lm.train_ds.vocab, bs=32, num_workers = 0)

In [5]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [7]:
data_lm = load_data(path2, 'data_lm_export.pkl')
data_clas = load_data(path2, 'data_clas_export.pkl', bs=16)

In [8]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.266793,4.029991,0.257807,08:37


In [9]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.014753,3.904624,0.273592,16:11


In [10]:
learn.predict("This is a review about", n_words=10)

"This is a review about the topic . i 'm not saying . Yes"

In [11]:
learn.save_encoder('ft_enc')

In [12]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

In [13]:
data_clas.show_batch()

text,target
"xxbos xxmaj please call me xxmaj xxunk . i 'm not quite sure how to introduce myself here . xxmaj there was this really odd thing that happened to me recently . i ca n't quite explain what happened ; i can only tell the bits that i remember clearly . xxmaj up until now , i 've only shared this with a very few close , trusted friends ,",infp
"xxbos xxup xxunk : xxmaj bernie xxmaj sanders : xxup yes , xxmaj xxunk $ xxunk billion for overseas contingency operations for fiscal year 2014 ( xxmaj xxunk . xxunk , xxunk , xxunk , xxunk & xxunk ) . xxmaj as well as removing due process giving the government total control over who gets due process & who does n't . xxmaj it permits the military to xxunk individuals",infp
"xxbos xxmaj so i 've always been fascinated by the study of nonverbal communication , and as an xxup enfj that loves to connect and engage with people , i found myself doing a lot of things unconsciously to speed up a new connection with someone . xxmaj and when i tried using them consciously during certain conversations , i found a lot of success . xxmaj and i definitely",enfj
"xxbos xxunk , to answer your question : i think ( as i believe you 're implying , too ) that xxmaj bernie and xxmaj trump are very different ... in many ways , including personality type . \r\r \n xxmaj but i 'd be willing to wager everything i 'd ever earn that neither xxmaj bernie nor xxmaj trump is an xxup entj . i have my impressions",infp
"xxbos xxmaj again very difficult to read , i love him so much and i do recognise some of our dynamics in what you 're say . i 'm very aware that he sometimes feels restricted , misunderstood and undervalued . i do hold my tongue sometimes in an effort to not seem judgmental , someone in his life is also an xxup enfj and he does find them judgemental",enfj


In [14]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.572555,0.544257,0.725554,11:19


In [15]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.576220,0.527515,0.740801,11:22


In [15]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time


RuntimeError: CUDA out of memory. Tried to allocate 40.50 MiB (GPU 0; 4.00 GiB total capacity; 2.95 GiB already allocated; 8.64 MiB free; 69.70 MiB cached)